In [1]:
import sys
import os
import gc


from utils import *
import pandas as pd
import numpy as np

path_build = "./build/"
path_embed = "./embed/"
path_list = "./feature_series/"
path_mnt = "./mnt/"
#path_mnt = "./mnt_train/"
#path_mnt = "./mnt_train_mean/"

In [2]:
#cos_to_local("/home/tione/notebook/build/click_log.pkl", "lht-build-1259079995", "click_log.pkl")
user = pd.read_pickle(path_build + "train_user.pkl")
click_log = pd.read_pickle(path_build + "click_log.pkl")
train_click_log = click_log[click_log['user_id']<=3000000]

In [3]:
ctr_df = train_click_log.merge(user, how='left', on='user_id')
ctr_df = pd.concat([ctr_df, pd.get_dummies(ctr_df['age'],prefix='age'), pd.get_dummies(ctr_df['gender'],prefix='gender')], axis=1)
del ctr_df['gender_2']

gender_p = dict(user['gender'].value_counts() / 3000000)
age_p = dict(user['age'].value_counts() / 3000000)
print(gender_p, age_p)

ctr_list = ['gender_1', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'age_10']
del user
gc.collect()

{1: 0.669494, 2: 0.330506} {3: 0.22545666666666667, 4: 0.166868, 2: 0.16568933333333333, 5: 0.14569333333333334, 6: 0.11322033333333334, 7: 0.07424233333333333, 1: 0.03906433333333333, 8: 0.035412, 9: 0.021577333333333334, 10: 0.012776333333333334}


0

In [4]:
numeric_features = ['product_id','product_category', 'advertiser_id', 'industry']
#numeric_features = [ 'ad_id', 'product_id', 'product_category', 'advertiser_id', 'industry']
mnt_df = pd.DataFrame()
mnt_df['user_id'] = range(1,4000001)
for fea in numeric_features:
    print(fea)
    creative_id_ctr = ctr_df.drop_duplicates(['user_id',fea])

    creative_id_ctr_df = creative_id_ctr.groupby([fea])[ctr_list].mean().add_prefix(f'prob_{fea}_').reset_index()
    creative_id_sum = creative_id_ctr.groupby([fea])[ctr_list].sum().add_prefix(f'sum_{fea}_').reset_index()
    creative_id_size = creative_id_ctr.groupby([fea]).size().reset_index().rename({0:'size'}, axis=1)
    creative_id_ctr_df[f'size'] = creative_id_size['size']
    #贝叶斯平滑
    for f in ctr_list:
        M = creative_id_ctr_df[f'prob_{fea}_{f}'].mean()
        S = creative_id_ctr_df[f'prob_{fea}_{f}'].var()
        alpha =( M * ((M*(1-M))/(S)-1)  )*2
        beta  =( (1-M) * ((M*(1-M))/(S)-1) )*2
        creative_id_ctr_df[f'sum_{fea}_{f}'] = creative_id_sum[f'sum_{fea}_{f}']
        creative_id_ctr_df[f'prob_{fea}_{f}'] = (creative_id_ctr_df[f'sum_{fea}_{f}'] + alpha) / (creative_id_ctr_df[f'size'] + alpha + beta)


    click_log_f = click_log.merge(creative_id_ctr_df, how='left', on=fea)

    #填充缺失值
    click_log_f[f'prob_{fea}_gender_1'] = click_log_f[f'prob_{fea}_gender_1'].fillna(gender_p[1])
    for i in range(1,11):
        click_log_f[f'prob_{fea}_age_{i}'] = click_log_f[f'prob_{fea}_age_{i}'].fillna(age_p[i])

    creative_id = pd.concat([
        click_log_f.groupby(['user_id'])[ [f'prob_{fea}_'+i for i in ctr_list] ].mean().add_prefix(f'mean_').reset_index(),
        click_log_f.groupby(['user_id'])[ [f'prob_{fea}_'+i for i in ctr_list] ].max().add_prefix(f'max_').reset_index().drop(['user_id'], axis=1),
        click_log_f.groupby(['user_id'])[ [f'prob_{fea}_'+i for i in ctr_list] ].min().add_prefix(f'min_').reset_index().drop(['user_id'], axis=1),
        click_log_f.groupby(['user_id'])[ [f'prob_{fea}_'+i for i in ctr_list] ].var().add_prefix(f'var_').reset_index().drop(['user_id'], axis=1)  
    ],axis=1)
    
    mnt_df = pd.concat([mnt_df, creative_id.drop(['user_id'], axis=1)], axis=1)

print(mnt_df.columns)
mnt_df.head()

product_id
product_category
advertiser_id
industry
Index(['user_id', 'mean_prob_product_id_gender_1',
       'mean_prob_product_id_age_1', 'mean_prob_product_id_age_2',
       'mean_prob_product_id_age_3', 'mean_prob_product_id_age_4',
       'mean_prob_product_id_age_5', 'mean_prob_product_id_age_6',
       'mean_prob_product_id_age_7', 'mean_prob_product_id_age_8',
       ...
       'var_prob_industry_age_1', 'var_prob_industry_age_2',
       'var_prob_industry_age_3', 'var_prob_industry_age_4',
       'var_prob_industry_age_5', 'var_prob_industry_age_6',
       'var_prob_industry_age_7', 'var_prob_industry_age_8',
       'var_prob_industry_age_9', 'var_prob_industry_age_10'],
      dtype='object', length=177)


,user_id,mean_prob_product_id_gender_1,mean_prob_product_id_age_1,mean_prob_product_id_age_2,mean_prob_product_id_age_3,mean_prob_product_id_age_4,mean_prob_product_id_age_5,mean_prob_product_id_age_6,mean_prob_product_id_age_7,mean_prob_product_id_age_8,...,var_prob_industry_age_1,var_prob_industry_age_2,var_prob_industry_age_3,var_prob_industry_age_4,var_prob_industry_age_5,var_prob_industry_age_6,var_prob_industry_age_7,var_prob_industry_age_8,var_prob_industry_age_9,var_prob_industry_age_10
0,1,0.672404,0.047073,0.169306,0.219687,0.161341,0.142774,0.111205,0.074686,0.036028,...,0.000083,0.000460,0.000570,0.000234,0.000067,0.000131,0.000155,0.000122,0.000125,0.000087
1,2,0.707452,0.046580,0.167935,0.213173,0.155749,0.138401,0.111276,0.075765,0.040921,...,0.000217,0.000849,0.000294,0.000064,0.000133,0.000155,0.000128,0.000068,0.000071,0.000073
2,3,0.554122,0.047793,0.158955,0.202371,0.156107,0.149734,0.123931,0.082371,0.039025,...,0.000245,0.000726,0.000175,0.000100,0.000124,0.000168,0.000188,0.000077,0.000044,0.000045
3,4,0.647952,0.037690,0.144504,0.199130,0.163411,0.162862,0.135022,0.084462,0.038399,...,0.000115,0.000375,0.000159,0.000045,0.000056,0.000105,0.000112,0.000049,0.000034,0.000028
4,5,0.692815,0.043466,0.164682,0.217611,0.161995,0.143558,0.113823,0.077853,0.038550,...,0.000183,0.000731,0.000190,0.000163,0.000148,0.000175,0.000120,0.000026,0.000015,0.000015


In [7]:
#mnt_df.to_pickle(f"{path_mnt}/user_mnts.pkl")
mnt_df.columns

reduce_mem_usage(mnt_df)


Mem. usage decreased to 1358.03 Mb (74.9% reduction)


In [8]:
mnt_df.to_pickle(f"{path_mnt}/user_mnts.pkl")
